## Import libraries, modules and data

In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras

In [ ]:
!pip install nltK

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models,layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from sklearn.preprocessing import LabelEncoder
df=pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin1',header=None)
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [2]:


#The data has no column headers,
df=df.drop(columns=[1,2,3,4],axis=1)

In [3]:
df["text"] = df[5]
df = df.drop(columns=[5],axis=1)
df

,0,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
...,...,...
1599995,4,Just woke up. Having no school is the best fee...
1599996,4,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
import re
import string

## Cleaning the data set

In [5]:
# Some basic helper functions to clean text by removing urls, emojis, html tags and punctuations.
def remove_stopwords(text):
    tokens = []
    for token in text.split():
        if token not in stop:
            tokens.append(token)
    return " ".join(tokens)


def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)


def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)


def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

# Applying helper functions
df['text_clean'] = df['text'].apply(lambda x: remove_stopwords(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_URL(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_html(x))
df['text_clean'] = df['text_clean'].apply(lambda x: remove_punct(x))

In [6]:
df.columns=['sentiment','tweets','tweets_clean']
df.head()

,sentiment,tweets,tweets_clean
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot Awww thats bummer You shoulda got...
1,0,is upset that he can't update his Facebook by ...,upset cant update Facebook texting it might cr...
2,0,@Kenichan I dived many times for the ball. Man...,Kenichan I dived many times ball Managed save ...
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass no behaving all im mad here I ...


In [7]:

data1=df[df['sentiment']==0].sample(2500)
data2=df[df['sentiment']==4].sample(2500)

In [8]:
df=data1.append(data2)
text=df['tweets'].to_list()

## Tokenization
Tokenization is a way of separating a piece of text into smaller units called tokens. Here, tokens can be either words, characters, or subwords.

In [9]:
token=Tokenizer()
token.fit_on_texts(text)
len(token.index_word)
vocab_size=len(token.word_index)+1
encoded_text=token.texts_to_sequences(text)
max_length=200
x=sequence.pad_sequences(encoded_text,maxlen=max_length,padding='pre')
x

array([[   0,    0,    0, ..., 1023, 1384,  248],
       [   0,    0,    0, ...,   60,  899,  249],
       [   0,    0,    0, ...,   11,    3, 1385],
       ...,
       [   0,    0,    0, ...,   27,   10,    7],
       [   0,    0,    0, ...,  608,  636,    8],
       [   0,    0,    0, ...,   28,    4, 3598]])

In [10]:
y=df['sentiment']
y=y.map({0:0,4:1})
y.shape

(5000,)

In [11]:
vec_size=500
nn=models.Sequential([
    
    layers.Embedding(vocab_size,vec_size,input_length=max_length),
    layers.Conv1D(64,8,activation='relu'),
    layers.MaxPooling1D(2),
    layers.Dropout(0.8),
    
    layers.Dense(40,activation='relu'),
    layers.Dropout(0.9),
    
    layers.Dense(29,activation='relu'),
    layers.GlobalMaxPooling1D(),
    
    layers.Dense(1,activation='sigmoid')
    
    
    ])

In [12]:
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 500)          5843500   
                                                                 
 conv1d (Conv1D)             (None, 193, 64)           256064    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 96, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 96, 64)            0         
                                                                 
 dense (Dense)               (None, 96, 40)            2600      
                                                                 
 dropout_1 (Dropout)         (None, 96, 40)            0         
                                                        

In [13]:


x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)
nn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
nn.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=25)



Epoch 1/25
125/125 [==============================] - 14s 110ms/step - loss: 0.7122 - accuracy: 0.5010 - val_loss: 0.6939 - val_accuracy: 0.5160
Epoch 2/25
125/125 [==============================] - 14s 109ms/step - loss: 0.7011 - accuracy: 0.5117 - val_loss: 0.6931 - val_accuracy: 0.5140
Epoch 3/25
125/125 [==============================] - 13s 108ms/step - loss: 0.6968 - accuracy: 0.5253 - val_loss: 0.6917 - val_accuracy: 0.5180
Epoch 4/25
125/125 [==============================] - 13s 107ms/step - loss: 0.6920 - accuracy: 0.5330 - val_loss: 0.6891 - val_accuracy: 0.5210
Epoch 5/25
125/125 [==============================] - 14s 109ms/step - loss: 0.6584 - accuracy: 0.6047 - val_loss: 0.6764 - val_accuracy: 0.5740
Epoch 6/25
125/125 [==============================] - 14s 108ms/step - loss: 0.6109 - accuracy: 0.6710 - val_loss: 0.6563 - val_accuracy: 0.6290
Epoch 7/25
125/125 [==============================] - 14s 109ms/step - loss: 0.5130 - accuracy: 0.7530 - val_loss: 0.6505 - val_ac

In [14]:
nn.evaluate(x_test,y_test)

32/32 [==============================] - 0s 12ms/step - loss: 0.7405 - accuracy: 0.6190


[0.7405259609222412, 0.6190000176429749]

## Predict function to be used in the telegram bot
A negative sentiment result would return false

In [61]:
def predict(x):
    remove_stopwords(x)
    remove_URL(x)
    remove_html(x)
    remove_punct(x)
    encoded_x=token.texts_to_sequences([x])
    x=pad_sequences(encoded_x,maxlen=max_length,padding='pre')
    print(nn.predict(x)>0.5)

In [62]:
predict("I HATE YOUR PRODUCTS")

1/1 [==============================] - 0s 17ms/step
[[False]]


In [65]:
nn.evaluate(x_test, y_test)

32/32 [==============================] - 0s 12ms/step - loss: 0.7405 - accuracy: 0.6190


[0.7405259609222412, 0.6190000176429749]

## Insert next data set for classifier model

In [66]:
dataset = pd.read_json('News_Category_Dataset_v3.json', lines=True)
dataset.drop(['authors', 'link', 'date'], axis = 1, inplace = True) 
dataset.head()

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...


In [67]:
dataset['category'].value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [68]:
politics_list = list(dataset[dataset['category'] == 'POLITICS'].index) 
list_16 = politics_list[:16000]
dataset2 = dataset.copy()
dataset2.drop(list_16, axis=0, inplace=True)
#politics has too many, need to drop

In [69]:
dataset2

,headline,category,short_description
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha..."
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to..."
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...
...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr..."
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...


In [70]:
dataset2["combined_text"] = dataset2["headline"]+dataset2["short_description"]
dataset2

,headline,category,short_description,combined_text
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li..."
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...
...,...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",Maria Sharapova Stunned By Victoria Azarenka I...
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...","Giants Over Patriots, Jets Over Colts Among M..."
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,Aldon Smith Arrested: 49ers Linebacker Busted ...


## Cleaning data using same helper functions

In [71]:
dataset2['text_clean'] = dataset2['combined_text'].apply(lambda x: remove_stopwords(x))
dataset2['text_clean'] = dataset2['text_clean'].apply(lambda x: remove_URL(x))
dataset2['text_clean'] = dataset2['text_clean'].apply(lambda x: remove_html(x))
dataset2['text_clean'] = dataset2['text_clean'].apply(lambda x: remove_punct(x))


In [72]:

dataset2['combined_text'][0]

'Over 4 Million Americans Roll Up Sleeves For Omicron-Targeted COVID BoostersHealth experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.'

In [73]:
dataset2

,headline,category,short_description,combined_text,text_clean
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li...",American Airlines Flyer Charged Banned For Lif...
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...,23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...,The Funniest Tweets From Parents This Week Sep...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...,Woman Who Called Cops On Black BirdWatcher Los...
...,...,...,...,...,...
209522,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,RIM CEO Thorsten Heins Significant Plans For B...
209523,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",Maria Sharapova Stunned By Victoria Azarenka I...,Maria Sharapova Stunned By Victoria Azarenka I...
209524,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...","Giants Over Patriots, Jets Over Colts Among M...",Giants Over Patriots Jets Over Colts Among Mos...
209525,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,Aldon Smith Arrested: 49ers Linebacker Busted ...,Aldon Smith Arrested 49ers Linebacker Busted F...


In [74]:
dataset3 = dataset2.drop(columns=['headline','short_description','combined_text'])


## Here we group the category as 42 is too many and too specific

In [75]:


categories = dataset3['category'].value_counts().index

def groupper(grouplist,name):
    for ele in categories:
        if ele in grouplist:
            dataset3.loc[dataset3['category'] == ele, 'category'] = name

groupper( grouplist= ['WELLNESS', 'HEALTHY LIVING','HOME & LIVING','STYLE & BEAUTY' ,'STYLE'] , name =  'LIFESTYLE AND WELLNESS')

groupper( grouplist= [ 'PARENTING', 'PARENTS' ,'EDUCATION' ,'COLLEGE'] , name =  'PARENTING AND EDUCATION')

groupper( grouplist= ['SPORTS','ENTERTAINMENT' , 'COMEDY','WEIRD NEWS','ARTS'] , name =  'SPORTS AND ENTERTAINMENT')

groupper( grouplist= ['TRAVEL', 'ARTS & CULTURE','CULTURE & ARTS','FOOD & DRINK', 'TASTE'] , name =  'TRAVEL-TOURISM & ART-CULTURE')

groupper( grouplist= ['WOMEN','QUEER VOICES', 'LATINO VOICES', 'BLACK VOICES'] , name =  'EMPOWERED VOICES')

groupper( grouplist= ['BUSINESS' ,  'MONEY'] , name =  'BUSINESS-MONEY')

groupper( grouplist= ['THE WORLDPOST' , 'WORLDPOST' , 'WORLD NEWS','U.S. NEWS'] , name =  'WORLDNEWS')

groupper( grouplist= ['ENVIRONMENT' ,'GREEN'] , name =  'ENVIRONMENT')

groupper( grouplist= ['TECH', 'SCIENCE'] , name =  'SCIENCE AND TECH')

groupper( grouplist= ['FIFTY' , 'IMPACT' ,'GOOD NEWS','CRIME'] , name =  'GENERAL')

groupper( grouplist= ['WEDDINGS', 'DIVORCE',  'RELIGION','MEDIA'] , name =  'MISC')



In [76]:
dataset3

,category,text_clean
0,WORLDNEWS,Over 4 Million Americans Roll Up Sleeves For O...
1,WORLDNEWS,American Airlines Flyer Charged Banned For Lif...
2,SPORTS AND ENTERTAINMENT,23 Of The Funniest Tweets About Cats And Dogs ...
3,PARENTING AND EDUCATION,The Funniest Tweets From Parents This Week Sep...
4,WORLDNEWS,Woman Who Called Cops On Black BirdWatcher Los...
...,...,...
209522,SCIENCE AND TECH,RIM CEO Thorsten Heins Significant Plans For B...
209523,SPORTS AND ENTERTAINMENT,Maria Sharapova Stunned By Victoria Azarenka I...
209524,SPORTS AND ENTERTAINMENT,Giants Over Patriots Jets Over Colts Among Mos...
209525,SPORTS AND ENTERTAINMENT,Aldon Smith Arrested 49ers Linebacker Busted F...


## Machine learning model using LSTM and glove6b embedding

In [77]:
X,Y = dataset3['text_clean'],dataset3['category']

#80% to train , 10% for validation , 10% for testing
X_train, X_val, y_train, y_val = train_test_split(X,Y, test_size=0.2, random_state=42)
X_val, X_test , y_val, y_test= train_test_split(X_val,y_val, test_size=0.5, random_state=42)

In [78]:
vocab_size =20000
max_length = 150
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(X_train,maxlen= max_length,padding=padding_type, truncating=trunc_type)
y_train = np.asarray(y_train)
y_train = pd.get_dummies(y_train)

X_val = tokenizer.texts_to_sequences(X_val)
X_val = sequence.pad_sequences(X_val,maxlen= max_length,padding=padding_type, truncating=trunc_type)
y_val = np.asarray(y_val)
y_val = pd.get_dummies(y_val)

train_set = np.array(X_train)
val_set = np.array(X_val)

train_label = np.array(y_train)
val_label = np.array(y_val)


y_test = pd.get_dummies(y_test)
y_test = np.asarray(y_test)
y_test = np.argmax(y_test,axis=1) 

print(train_set.shape)
print(train_label.shape)


print(val_set.shape)
print(val_label.shape)

(154821, 150)
(154821, 12)
(19353, 150)
(19353, 12)


In [80]:
path_to_glove_file =  './glove.6B.100d.txt'

In [82]:
#Initialising the embedding matrix with glove vec embeddings

num_tokens = len(tokenizer.word_index.items()) + 2
embedding_dim = 100
hits = 0
misses = 0


embeddings_index = {}
with open(path_to_glove_file, encoding="utf8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 67098 words (139106 misses)


In [83]:
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                            patience=3, min_delta=0.0001)


tf.keras.backend.clear_session()
embed_size = 100
model = keras.models.Sequential([
                                 
        Embedding(num_tokens,
        embedding_dim,
        embeddings_initializer=keras.initializers.Constant(embedding_matrix),
        mask_zero=True,input_shape=[None],trainable=False),
        keras.layers.Bidirectional(keras.layers.LSTM(256, dropout = 0.4)),
        keras.layers.Dense(12, activation="softmax")
            
        ])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         20620600  
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 12)                6156      
                                                                 
Total params: 21,357,892
Trainable params: 737,292
Non-trainable params: 20,620,600
_________________________________________________________________


In [84]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
history = model.fit( train_set,train_label,
                     batch_size = 32,
                     steps_per_epoch=len(X_train) // 32, 
                     validation_data = (val_set , val_label),
                     validation_steps = len(val_set)//32, epochs=1,
                     callbacks=  early_stop )

4838/4838 [==============================] - 5404s 1s/step - loss: 1.2494 - accuracy: 0.6008 - val_loss: 1.0918 - val_accuracy: 0.6539


## Prediction function that returns both the category and the sentiment

In [85]:
def predictboth(x):
    remove_stopwords(x)
    remove_URL(x)
    remove_html(x)
    remove_punct(x)
    x=token.texts_to_sequences([x])
    x=sequence.pad_sequences(x,maxlen=max_length,padding='pre')
    result = model.predict(x)[0]
    index = np.argmax(result,axis=0)
    cat = ['LIFESTYLE AND WELLNESS','POLITICS','PARENTING AND EDUCATION','SPORTS AND ENTERTAINMENT','TRAVEL-TOURISM & ART-CULTURE','EMPOWERED VOICES','BUSINESS-MONEY','WORLDNEWS','ENVIRONMENT','SCIENCE AND TECH','GENERAL','MISC']
    print(cat[index],nn.predict(x)>0.5)

In [105]:
predictboth("visit paris")

1/1 [==============================] - 0s 16ms/step
TRAVEL-TOURISM & ART-CULTURE [[ True]]
